In [1]:
from pathlib import Path
import os
import xmltodict
from xml.etree import ElementTree
import re
from tqdm import tqdm

In [2]:
base_path = Path("D:\game resourse Extract\PC\Golden Treasure The Great Green\Resources\Mods\\")

In [3]:
event_files = []
for sub_path in os.listdir(base_path):
    event_files.extend(
        base_path.joinpath(sub_path).joinpath('Events').joinpath(file)
        for file in os.listdir(base_path.joinpath(sub_path).joinpath('Events')) if file.endswith('.xml'))
event_files

[WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/actone.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/actone_l.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/actthree_1.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/actthree_2.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/actthree_3.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/actthree_4.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/actthree_l.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasure The Great Green/Resources/Mods/Main/Events/acttwo_1.xml'),
 WindowsPath('D:/game resourse Extract/PC/Golden Treasur

In [4]:
print(path:=base_path.joinpath('Main').joinpath('Events').joinpath('actone.xml'))

D:\game resourse Extract\PC\Golden Treasure The Great Green\Resources\Mods\Main\Events\actone.xml


In [5]:
def texts_from_events_dict(events_dict):
    tag_re = re.compile(r'<[^<]+?>|\[[^\]]+?]')
    texts_all = []

    event_ids_unvisited = set(events_dict.keys())
    queue = []

    while event_ids_unvisited:
        queue.append(next(iter(event_ids_unvisited)))

        while queue:
            event_id = queue.pop()
            if event_id not in event_ids_unvisited:
                continue
            event_ids_unvisited.remove(event_id)
            #print(event_id)
            if event_id not in events_dict:
                continue
            event = events_dict[event_id]
            if event['event_body']:
                texts = event['event_body']['text_break']
                if isinstance(texts, dict):
                    texts = [texts]
                for text in texts:
                    if text_raw:=text['text']:
                        text_raw = tag_re.sub('', text_raw)
                        if text_raw.startswith('/') or '_' in text_raw:
                            continue
                        #print(text_raw)
                        texts_all.append(text_raw)

            if event['event_options']:
                options = event['event_options']['event_option']
                if isinstance(options, dict):
                    options = [options]

                for option in options:
                    if not option['option_text']:
                        continue
                    option_raw = tag_re.sub('', option['option_text'])
                    if option_raw.startswith('/') or '_' in option_raw:
                        continue
                    #print('-', option_raw)
                    texts_all.append(option_raw)
                    
                    if not option['destinations']:
                        continue
                    destinations = option['destinations']['destination']
                    if isinstance(destinations, dict):
                        destinations = [destinations]
                    for destination in destinations:
                        queue.append(destination['destination_id'])
    return texts_all

In [6]:
texts_all = []
for event_file_path in tqdm(event_files):
    with open(event_file_path, 'rb') as xml_file:
        events = xmltodict.parse(xml_file)
    events_dict = {e['event_name']: e for e in events['events']['event']}
    texts_all.extend(texts_from_events_dict(events_dict))
texts_all = [*dict.fromkeys(texts_all)]

100%|██████████| 26/26 [00:01<00:00, 13.01it/s]


In [7]:
len(texts_all)

8956

In [8]:
texts_all[1000:1000+10]

['The Salamanders look askance, tails limp in disappointment. Ah. Perhaps such things are beyond a hatchling such as yourself. And you have no news of our Enkindler, the Great Dark One, either, I suppose? No? Well, a visit from a Draak-Kin is a good omen, in any case.',
 'Have you further dealings with us, newborn Kin to the Great Ones?',
 'Ask about the Draak who Enkindled them.',
 'Ask about their Baretail servants.',
 'Ask them to share their wisdom with you.',
 'Ask about the strange blue fungus you have seen above.',
 'Challenge these Fire-touched beings for their great Treasure.',
 'Breathe out your flaming breath upon the Salamanders.',
 'Climb back up the shaft from which you came.',
 'The Salamanders thump their tails and roll about within your flames, clearly overjoyed. Ah, such pleasure! Many thanks, intones the largest.']

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 2_000_000

doc = nlp(' '.join(texts_all).replace('\n', ' ').lower())
len(doc)

204261

In [10]:
from collections import Counter

token_counter = Counter((token.lemma_.lower(), token.pos_) for token in doc if token.lower_ not in nlp.Defaults.stop_words)
for token, token_pos in list(token_counter):
    if all(c in ',;:.!?-"()[]{}\n ' for c in token):
        del token_counter[token, token_pos]

print('Number of unique tokens:', len(token_counter))
for (token, token_pos), count in token_counter.most_common(200):
    print((token, token_pos), end=', ')

Number of unique tokens: 8133
('great', 'ADJ'), ('time', 'NOUN'), ('body', 'NOUN'), ('know', 'VERB'), ('way', 'NOUN'), ('shall', 'AUX'), ('kin', 'PROPN'), ('mind', 'NOUN'), ('song', 'NOUN'), ('sun', 'NOUN'), ('earth', 'NOUN'), ('thing', 'NOUN'), ('treasure', 'NOUN'), ('essence', 'NOUN'), ('tail', 'NOUN'), ('leave', 'VERB'), ('sing', 'VERB'), ('away', 'ADV'), ('find', 'VERB'), ('true', 'ADJ'), ('come', 'VERB'), ('life', 'NOUN'), ('like', 'ADP'), ('dance', 'NOUN'), ('draak', 'NOUN'), ('feel', 'VERB'), ('destruction', 'NOUN'), ('water', 'NOUN'), ('new', 'ADJ'), ('small', 'ADJ'), ('destroy', 'VERB'), ('being', 'NOUN'), ('fire', 'NOUN'), ('people', 'NOUN'), ('learn', 'VERB'), ('begin', 'VERB'), ('truth', 'NOUN'), ('tree', 'NOUN'), ('scent', 'NOUN'), ('place', 'NOUN'), ('green', 'NOUN'), ('lair', 'NOUN'), ('flesh', 'NOUN'), ('spirit', 'NOUN'), ('air', 'NOUN'), ('return', 'VERB'), ('stone', 'NOUN'), ('land', 'NOUN'), ('power', 'NOUN'), ('strong', 'ADJ'), ('kin', 'NOUN'), ('ask', 'VERB'), ('of

In [11]:
parts_of_speach = list(set(pos for token, pos in token_counter))
print(parts_of_speach)

['PROPN', 'INTJ', 'X', 'DET', 'PART', 'ADV', 'CCONJ', 'ADJ', 'SCONJ', 'SPACE', 'PUNCT', 'VERB', 'ADP', 'NOUN', 'AUX', 'SYM', 'NUM', 'PRON']


In [12]:
token_counter_by_pos = {
    pos: Counter({token: count for (token, pos_), count in token_counter.items() if pos == pos_}) 
    for pos in parts_of_speach}

In [13]:
parts_of_speach.sort(key=lambda pos: len(token_counter_by_pos[pos]), reverse=True)
for pos in parts_of_speach:
    print(f'> {pos}: ')
    print(', '.join(text for text, count in token_counter_by_pos[pos].most_common(200) if count > 1))

> NOUN: 
time, body, way, mind, song, sun, earth, thing, treasure, essence, tail, life, dance, draak, destruction, water, being, fire, people, truth, tree, scent, place, green, lair, flesh, spirit, air, stone, land, power, kin, world, kind, tailless, self, eye, territory, goodbeast, pain, head, strength, beast, elder, ground, fear, feather, wisdom, flame, light, shinestone, end, vantage, clansinger, knowledge, music, voice, prey, path, clan, hatchling, moment, warrior, breath, question, core, child, birther, night, creation, turning, reality, future, creator, challenge, heart, day, home, tribe, desire, dream, trove, giver, joy, sense, mouth, den, presence, element, claw, tradition, void, form, law, victory, star, sound, gift, onesong, wing, love, wind, artifact, thought, woodstrider, object, sky, chamber, bloom, tooth, meal, school, seed, fang, nature, blessing, darkness, blazetail, grass, fact, number, skin, silence, surefoot, spiritwood, secret, master, food, moon, hive, enemy, wealt

In [27]:
for sent in [ent.sent for ent in doc if ent.lower_.endswith('jade')][:20]:
    print('>', sent)

> in a far riseward realm, all kin lower themselves in the presence of jade cloud, and the fiery islands beyond fear the raging glory of bladetail-of-the-several-minds.
> even jade cloud, the soft-sung yet undisputed master of the riseward lands, has come.


In [15]:
matcher = spacy.matcher.Matcher(nlp.vocab)
matcher.add('DANCES', [[{'LOWER': 'dance'}, {'LOWER': 'of'}, {'OP': '?'} , {'POS': {'IN': ['NOUN', 'PROPN']}}]])
matcher.add('SONGS', [[{'LOWER': 'song'}, {'LOWER': 'of'}, {'OP': '?'} , {'POS': {'IN': ['NOUN', 'PROPN']}}]])
matches = matcher(doc)
Counter((str(doc[start:start+1]).lower(), str(doc[start:end]).lower()) for _, start, end in matches).most_common()

[(('dance', 'dance of destruction'), 62),
 (('dance', 'dance of creation'), 25),
 (('song', 'song of annunciation'), 8),
 (('song', 'song of destruction'), 7),
 (('song', 'song of healing'), 7),
 (('song', 'song of thanksgiving'), 5),
 (('song', 'song of challenge'), 5),
 (('song', 'song of dominion'), 4),
 (('song', 'song of joy'), 3),
 (('song', 'song of victory'), 3),
 (('song', 'song of defiance'), 3),
 (('song', 'song of praise'), 3),
 (('song', 'song of transformation'), 3),
 (('dance', 'dance of submission'), 3),
 (('song', 'song of farewell'), 2),
 (('song', 'song of protest'), 2),
 (('song', 'song of aid'), 2),
 (('dance', 'dance of the cosmos'), 1),
 (('song', 'song of sorts'), 1),
 (('song', 'song of your life'), 1),
 (('dance', 'dance of light sings'), 1),
 (('song', 'song of objection'), 1),
 (('dance', 'dance of great destruction'), 1),
 (('song', 'song of domination'), 1),
 (('song', 'song of command'), 1),
 (('song', 'song of surrender'), 1),
 (('song', 'song of the cla

In [32]:
from nltk.corpus import wordnet as wn

[ss for s in wn.synsets('cerulean') for sp in s.hypernym_paths() for ss in sp]

[Synset('entity.n.01'),
 Synset('abstraction.n.06'),
 Synset('attribute.n.02'),
 Synset('property.n.02'),
 Synset('visual_property.n.01'),
 Synset('color.n.01'),
 Synset('chromatic_color.n.01'),
 Synset('blue.n.01'),
 Synset('azure.n.01'),
 Synset('azure.s.01')]

In [36]:
def is_color(word):
    synsets = [ss for s in wn.synsets(word) for sp in s.hypernym_paths() for ss in sp]
    return wn.synset('chromatic_color.n.01') in synsets

In [37]:
sents = [str(ent.sent) for ent in doc if is_color(ent.lower_)]
sents = [*dict.fromkeys(sents)]
print(len(sents))
for sent in sents:
    print(sent)

507
you know, however, that you must leave soon; at that time, you will return here, and take a fateful step into the great green.
the great green shall belong to you, and you shall belong to it; so it is sung by the music in your heart.
if you lose, you will cede all of your treasure to twist and leave the great green forever, but if you win, this land shall become yours.
today, five clans of the draak-kin tribe remain: your clan, the emerald kin, who dwell mostly in the green; my clan, the sapphire kin, who make their lairs in the blue, where the spirit of water is sovereign; the ruby kin, who dwell in earth's flesh, close to the secret fire within her breast; the onyx kin, scions of the void, whose ways are shrouded in mystery; and the crystal kin, of whom very few remain.
ask darktooth about the blue fungus.
(15 treasures) offer darktooth the amber circle.
ask darktooth to purge the blue-blight from your body himself, expensive and painful though it be.
(5 treasures) show darktooth